# Computational Bionics: Project Exercise 1

Group members:
- Yash Waware
- Julian Lork
- Florian Frech (12308544)

## Task 1: Modeling the Leg

### 1.1 Description

The following section consider the model of the leg, described by an ODE model.

A test person with a body mass of $m_b = 100$ kg is chosen.

Following state variables are involved:
- $q_1$: Extension angle of the **hip** around then transversal axis ($xy$-plane)
- $\omega_1$: Angular velocity of the tight
- $q_2$: Rotation angle of the **knee** around the transversal axis ($xy$-plane) 
- $\omega_2$ Angular velocity of the shank

The segement masses are considered as point masses in the middle of each segment.

![](LegModel.png)

|Variable|Description|
|--------|-----------|
|$r_H$|Position vector of the hip to the reference system|
|$H$|Hip Joint|
|$K$|Knee Joint|
|$O$|Tight|
|$U$|Shank|
|$S$|Centre of Mass of the Foot|
|$SO$|Centre of Mass of the Tight|
|$SU$|Centre of Mass Shank|
|$m_1$|Tight Mass|
|$m_2$|Shank Mass|
|$m_3$|Foot Point Mass|
|$J_1$|Moment of Inertia of the Tight|
|$J_2$|Moment of Inertia of the Shank|
|$\alpha_H$|Flexion / Extension Angle of the Hip|
|$\alpha_K$|Flexion / Extension Angle of the Knee|


Following relations for the partial masses of the legs are assumed:

|Segment| Mass m [kg]|
|------|--------------|
|Tight|  $m_1 = 0.090 \cdot m_b + 0.73 = 9.73$|
|Shank| $m_2 = 0.055 \cdot m_b - 0.43 = 5.07$|
|Foot| $m_3 = 0.001 \cdot m_b + 0.34 = 0.44$|

### 1.2 Simplified Box Model

<!-- ![](BoxModelLegModel.png) -->

**Input:**
- anatomical data
- segment weights
- proportions of the limbs
- length and position of the segments
- gait data for one cycle
- Flexion / Extension **angles** of the ankle, knee, and hip joint

**Equations:**

**Output:**
- Ground reaction **forces** in y- and x- direction
- **Moments** around foot, knee, and hip joints


### 1.3 Setting
- Length of the Tight: $l_O = 0.410$ m
- Length of the Shank: $l_U = 0.415$ m

https://personal.cityu.edu.hk/meachan/Online%20Anthropometry/Chapter2/Ch2-5.htm

https://pmc.ncbi.nlm.nih.gov/articles/PMC5305206/

In [197]:
# Imports
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
import numpy as np
import sympy as sp
import pandas as pd
import matplotlib.pyplot as plt

In [198]:
# define numerical values
l_O_val = 0.410 # length of the tigh [m]
l_U_val = 0.415 # length of the shank [m]

m_body = 100 # mass of the body [kg]

mO_val = 9.73 # mass of the thigh [kg]
mU_val = 5.07 # mass of the shank [kg]
mF_val = 0.44 # mass of the foot [kg]

g_val = 9.81 # gravity [m/s^2]

In [222]:
# define symbolic variables
l_O, l_U, q1, q2, t = sp.symbols('l_O l_U q1 q2 t')
omega1, omega2 = sp.symbols(r'\omega_1 \omega_2')
omega1_dot, omega2_dot = sp.symbols(r'\dot{\omega}_1 \dot{omega}_2')
mO, mU, mF = sp.symbols('m_O m_U m_F')
g = sp.symbols('g')

# define angles q as a function of time
q1 = sp.Function('q1')(t)
q2 = sp.Function('q2')(t)

# define angular velocities w as the derivative of the angles
w1 = q1.diff(t)
w2 = q2.diff(t)

# define angular accelerations as the derivative of the angular velocities
dot_w1 = w1.diff(t)
dot_w2 = w2.diff(t)

### 1.4 Derivation 

**Relation between common measured joint angle and the chosen generalized coordinates**

$$q_1 = \alpha_H + \frac{3}{2}\pi$$

$$q_2 = q_1 - \alpha_K = \alpha_H + \frac{3}{2}\pi - \alpha_K$$

**Position vectors** to the centre of mass of tight $SO$, shank $SU$, and foot $S$ with respect to the hip $H$

https://scienceworld.wolfram.com/physics/DoublePendulum.html

https://en.wikipedia.org/wiki/Double_pendulum


$$
\mathbf{r}_{SO} = 0.5 \cdot l_O \cdot \begin{bmatrix} 
                   \cos q_1 \\
                   \sin q_1 \\
                   0 \end{bmatrix} \quad \quad

\mathbf{r}_{SU} = \begin{bmatrix}
                   l_O \cos q_1 + 0.5 \cdot l_U \cos q_2 \\
                   l_O \sin q_1 + 0.5 \cdot l_U \sin q_2 \\
                   0 \end{bmatrix} \quad \quad

\mathbf{r}_{S} = \begin{bmatrix}
                   l_O \cos q_1 + l_U \cos q_2 \\
                   l_O \sin q_1 + l_U \sin q_2 \\
                   0 \end{bmatrix} \quad \quad                   
$$

In [223]:
# Position vectors
r_SO = sp.Matrix([0.5 * l_O * sp.cos(q1), 0.5 * l_O *sp.sin(q1), 0])
r_SU = sp.Matrix([l_O * sp.cos(q1) + 0.5 * l_U * sp.cos(q2), l_O * sp.sin(q1) + 0.5 * l_U * sp.sin(q2), 0])
r_S = sp.Matrix([l_O * sp.cos(q1) + l_U * sp.cos(q2), l_O * sp.sin(q1) + l_U * sp.sin(q2), 0])

<!-- #![](SketchTask1.png) -->

**Velocity vectors** to the centre of mass of tight $SO$, shank $SU$, and foot $S$

$$
\mathbf{v}_{SO} = 0.5 \cdot l_O \cdot \dot{q}_1\begin{bmatrix} - \sin q_1 \\ \cos q_1 \\ 0 \end{bmatrix} = 0.5 \cdot l_O \cdot \omega_1\begin{bmatrix} - \sin q_1 \\ \cos q_1 \\ 0 \end{bmatrix}
$$

$$
\mathbf{v}_{SU} = l_O \cdot \dot{q}_1 \begin{bmatrix}  - \sin q_1 \\ \cos q_1 \\ 0 \end{bmatrix} + 0.5 \cdot l_U \cdot \dot{q}_2 \begin{bmatrix}  - \sin q_2 \\ \cos q_2 \\ 0 \end{bmatrix} =
 l_O \cdot \omega_1 \begin{bmatrix}  - \sin q_1 \\ \cos q_1 \\ 0 \end{bmatrix} + 0.5 \cdot l_U \cdot \omega_2 \begin{bmatrix}  - \sin q_2 \\ \cos q_2 \\ 0 \end{bmatrix}
$$

$$
\mathbf{v}_{S} = l_O \cdot \dot{q}_1 \begin{bmatrix}  - \sin q_1 \\ \cos q_1 \\ 0 \end{bmatrix} + l_U \cdot \dot{q}_2 \begin{bmatrix}  - \sin q_2 \\ \cos q_2 \\ 0 \end{bmatrix}
= l_O \cdot \omega_1 \begin{bmatrix}  - \sin q_1 \\ \cos q_1 \\ 0 \end{bmatrix} + l_U \cdot \omega_2 \begin{bmatrix}  - \sin q_2 \\ \cos q_2 \\ 0 \end{bmatrix}
$$

In [243]:
# Define velocity vectors
v_SO = r_SO.diff(t)
v_SU = r_SU.diff(t)
v_S = r_S.diff(t)

**Kinetic Energy T**

$$
T_1 = \frac{1}{2} m_O \mathbf{v}_{SO}^T \mathbf{v}_{SO}
$$

$$
T_2 = \frac{1}{2} m_U \mathbf{v}_{SU}^T \mathbf{v}_{SU}
$$

$$
T_3 = \frac{1}{2} m_F \mathbf{v}_{S}^T \mathbf{v}_{S}
$$

In [244]:
# Kinetic energy
T1 = 0.5 * mO * v_SO.dot(v_SO)
T2 = 0.5 * mU * v_SU.dot(v_SU)
T3 = 0.5 * mF * v_S.dot(v_S)
T = T1 + T2 + T3

**Potential Energy V**

$$
V_1 = m_O \cdot g \cdot 0.5 \cdot l_O \cdot \sin q_1
$$

$$
V_2 = m_U \cdot g \cdot ( l_O \sin q_1 + 0.5 \cdot l_U \sin q_2)
$$

$$
V_3 = m_F \cdot g \cdot ( l_O \sin q_1 + \cdot l_U \sin q_2)
$$

In [245]:
# Potential energy
V1 = mO * g * r_SO[1]
V2 = mU * g * r_SU[1]
V3 = mF * g * r_S[1]
V = V1 + V2 + V3

**Lagrange Function**

$$L = T - V$$

$$
L = T_1 + T_2 + T_3 - V_1 - V_2 - V_3
$$

$$
L = \frac{1}{2} \left[ m_O \mathbf{v}_{SO}^T \mathbf{v}_{SO} + m_U \mathbf{v}_{SU}^T \mathbf{v}_{SU} + m_F \mathbf{v}_{S}^T \mathbf{v}_{S} \right]
- g \cdot \left[ m_O \cdot 0.5 \cdot l_O \cdot \sin q_1 +  m_U \cdot ( l_O \sin q_1 + 0.5 \cdot l_U \sin q_2) + m_F \cdot ( l_O \sin q_1 + \cdot l_U \sin q_2) \right]
$$


**Approach:**
$$\frac{\partial}{\partial t} \left(\frac{\partial L}{\partial \dot{q}_i} - \frac{\partial L}{\partial q_i} \right) = Q_i$$

with $ \dot{q}_i = \omega_i$:

In [248]:
# Lagrangian
L = T - V

L = L.subs({q1.diff(t): omega1, q2.diff(t): omega2})
L

-0.5*g*l_O*m_O*sin(q1(t)) - g*m_F*(l_O*sin(q1(t)) + l_U*sin(q2(t))) - g*m_U*(l_O*sin(q1(t)) + 0.5*l_U*sin(q2(t))) + 0.5*m_F*((-\omega_1*l_O*sin(q1(t)) - \omega_2*l_U*sin(q2(t)))**2 + (\omega_1*l_O*cos(q1(t)) + \omega_2*l_U*cos(q2(t)))**2) + 0.5*m_O*(0.25*\omega_1**2*l_O**2*sin(q1(t))**2 + 0.25*\omega_1**2*l_O**2*cos(q1(t))**2) + 0.5*m_U*((-\omega_1*l_O*sin(q1(t)) - 0.5*\omega_2*l_U*sin(q2(t)))**2 + (\omega_1*l_O*cos(q1(t)) + 0.5*\omega_2*l_U*cos(q2(t)))**2)

In [ ]:
# Derivatives: dL / dw_i
dLdomega1 = sp.diff(L, omega1)
dLdomega2 = sp.diff(L, omega2)

d

In [ ]:
# Derivatives: d/dt (dL / dw_i)
ddt_dLdw1 = sp.diff(dLdw1, t)
ddt_dLdw2 = sp.diff(dLdw2, t)

# Derivatives: dL / dq_i
dLdq1 = sp.diff(L, q1)
dLdq2 = sp.diff(L, q2)

In [ ]:
# # Lagrangian
# L = T - V

# # Derivatives
# dLdw1 = sp.diff(L, w1).simplify()
# dLdw2 = sp.diff(L, w2).simplify()
# dLdq1 = sp.diff(L, q1).simplify()
# dLdq2 = sp.diff(L, q2).simplify()

# # Time derivatives of dL/dw_i
# 
# ddt_dLdw1 = sp.diff(dLdw1, q1) * w1 + sp.diff(dLdw1, q2) * w2 + sp.diff(dLdw1, w1) * w1_dot + sp.diff(dLdw1, w2) * w2_dot
# ddt_dLdw2 = sp.diff(dLdw2, q1) * w1 + sp.diff(dLdw2, q2) * w2 + sp.diff(dLdw2, w1) * w1_dot + sp.diff(dLdw2, w2) * w2_dot

In [194]:
# Equations of motion
eq1 = ddt_dLdw1 - dLdq1

eq2 = ddt_dLdw2 - dLdq2

# eq1 = eq1.simplify()
eq1

g*(l_O*m_F*cos(q1) + 0.5*l_O*m_O*cos(q1) + l_O*m_U*cos(q1)) + 1.0*l_O*l_U*m_F*w_1*w_2*sin(q1 - q2) + 0.5*l_O*l_U*m_U*w_1*w_2*sin(q1 - q2)

In [192]:
# Substitute constants
#eq1 = eq1.subs({l_O: l_O_val, l_U: l_U_val, mO: mO_val, mU: mU_val, mF: mF_val, g: g_val})
# eq2 = eq2.subs({l_O: l_O_val, l_U: l_U_val, mO: mO_val, mU: mU_val, mF: mF_val, g: g_val})

In [187]:
# Solve for angular accelerations
w1_dot_sol = sp.solve(eq1, w1_dot)[0]
w2_dot_sol = sp.solve(eq2, w2_dot)[0]

# Convert to numerical functions
domega1 = sp.lambdify((q1, q2, w1, w2), w1_dot_sol, 'numpy')
domega2 = sp.lambdify((q1, q2, w1, w2), w2_dot_sol, 'numpy')

In [188]:
w1_dot_sol

-0.379135094467092*w2_dot*cos(q1 - q2) - 4.15772056115389e-17*w_1*w_2*sin(q1 - q2) - 0.379135094467092*w_2**2*sin(q1 - q2) - 31.2547502245561*cos(q1)

In [183]:
domega1(0, 0, 0, 0)
domega2(0, 0, 0, 0)

-1.72132159678244*w1_dot - 41.1857679620385

In [171]:
# Read gait data
filename = 'gait_data.xls'
gait_data = pd.read_excel(filename, engine='xlrd')

# Extract gait data
gait_step = np.array(gait_data["gait_%"])
GRFz = np.array(gait_data["GRFz[%BW]"]) * m_body * g_val / 100
GRFx = gait_data["GRFx[%BW]"] * m_body * g_val / 100
MX_H = np.array(gait_data["MX_H[Nm/kg]"])
MX_K = np.array(gait_data["MX_K[Nm/kg]"])
q1_gait = np.deg2rad(np.array(gait_data["Flex_Ext_H[deg]"])) + 3 / 2 * np.pi
q2_gait = q1_gait - np.deg2rad(np.array(gait_data["Flex_Ext_K[deg]"]))

# Interpolate the moment data
MX_H_interp = interp1d(gait_step, MX_H, kind='cubic')
MX_K_interp = interp1d(gait_step, MX_K, kind='cubic')

# Time step
dt = 0.01
t_eval = np.arange(0, len(gait_step) * dt, dt)

In [173]:
#Initial conditions
q1_0 = q1_gait[0]
q2_0 = q2_gait[0]
omega1_0 = 0
omega2_0 = 0

In [179]:
# ODE system
def ode_system(t, y):
    q1, q2, omega1, omega2 = y

    dq1 = omega1
    dq2 = omega2
    domega1 = domega1(q1, q2, omega1, omega2)
    domega2 = domega2(q1, q2, omega1, omega2)

    return [dq1, dq2, domega1, domega2]

In [178]:
# Solve ODEs
sol = solve_ivp(ode_system, [0, len(gait_step) * dt], [q1_0, q2_0, omega1_0, omega2_0], t_eval=t_eval)

print("Done")

UnboundLocalError: cannot access local variable 'domega1' where it is not associated with a value

$$\frac{\partial}{\partial t} \left(\frac{\partial L}{\partial \dot{q}_1} - \frac{\partial L}{\partial q_1} \right) = Q_1 \quad \quad (I)$$

$$\frac{\partial}{\partial t} \left(\frac{\partial L}{\partial \dot{q}_2} - \frac{\partial L}{\partial q_2} \right) = Q_2 \quad \quad (II)$$

with $Q_1 = M_H$ and $Q_2 = M_K$

In [126]:
M_H, M_K, w1_dot, w2_dot  = sp.symbols(r'M_H M_K \dot{w}_1 \dot{w}_2')

# Equations of motion for q1 and q2
eq1 = ddt_dLdw1 - dLdq1 - M_H
eq2 = ddt_dLdw2 - dLdq2 - M_K

eq1 = eq1.subs(w1.diff(t), w1_dot).subs(w2.diff(t), w2_dot)
eq2 = eq2.subs(w2.diff(t), w2_dot).subs(w1.diff(t), w1_dot)

#eq1 = eq1.simplify()
#eq2 = eq2.simplify()

w1_dot = sp.solve(eq1, w1_dot,  simplify=True, rational=True)
w2_dot = sp.solve(eq2, w2_dot,  simplify=True, rational=True)

In [127]:
eq1

-M_H + g*(l_O*m_F*cos(q1) + 0.5*l_O*m_O*cos(q1) + l_O*m_U*cos(q1)) + 1.0*l_O*l_U*m_F*w_1*w_2*sin(q1 - q2) + 0.5*l_O*l_U*m_U*w_1*w_2*sin(q1 - q2)

----------------------------------------------------------

In [128]:
import numpy as np
import pandas as pd

In [140]:
# read gait file
filename = 'gait_data.xls'
gait_data = pd.read_excel(filename, engine='xlrd')

# Read Gait Data
gait_step = np.array(gait_data["gait_%"])
GRFz = np.array(gait_data["GRFz[%BW]"]) * m_body * g_val / 100
GRFx = gait_data["GRFx[%BW]"] * m_body * g_val / 100
MX_H = np.array(gait_data["MX_H[Nm/kg]"])
MX_K = np.array(gait_data["MX_K[Nm/kg]"])
q1_gait = np.deg2rad(np.array(gait_data["Flex_Ext_H[deg]"])) + 3 / 2 * np.pi
q2_gait = q1_gait - np.deg2rad(np.array(gait_data["Flex_Ext_K[deg]"]))

In [ ]:
# Interpolate the moment data
from scipy.interpolate import interp1d

MX_H = interp1d(gait_step, MX_H, kind='cubic')
MX_H = interp1d(gait_step, MX_K, kind='cubic')

In [148]:
# Time Step
dt = 0.01
t_eval = np.arange(0, len(gait_step) * dt, dt)

In [149]:
# Initial Conditions
q1_0 = q1_gait[0]
q2_0 = q2_gait[0]
omega1_0 = 0
omega2_0 = 0


In [152]:
# ODE System
def ode_system(t, y):
    q1, q2, omega1, omega2 = y
    


    domega1 = sp.lambdify((q1, q2, omega1, omega2, M_H, M_K), w1_dot, 'numpy')
    domega2 = sp.lambdify((q1, q2, omega1, omega2, M_H, M_K), w2_dot, 'numpy')

    dq1 = omega1
    dq2 = omega2

    return [dq1, dq2, domega1, domega2]


In [153]:
# Solve ODEs
sol = solve_ivp(ode_system, [0, len(gait_step) * dt], [q1_0, q2_0, omega1_0, omega2_0], t_eval=t_eval)

NotImplementedError: unhandled type: <class 'numpy.ndarray'>, [ 2.10394955e-01  5.99953076e-01  7.45118984e-01  6.94701285e-01
  6.06943325e-01  5.34739767e-01  4.42566788e-01  3.17943639e-01
  1.79433990e-01  4.99698840e-02 -4.65297432e-02 -1.08527246e-01
 -1.43472479e-01 -1.60752807e-01 -1.67495834e-01 -1.68664813e-01
 -1.68052566e-01 -1.67925323e-01 -1.72968512e-01 -1.78694776e-01
 -1.83769619e-01 -1.85151401e-01 -1.82157328e-01 -1.75516523e-01
 -1.65733609e-01 -1.55303890e-01 -1.44763574e-01 -1.36204856e-01
 -1.31452951e-01 -1.31395773e-01 -1.33618692e-01 -1.36860905e-01
 -1.40805536e-01 -1.46087786e-01 -1.51100513e-01 -1.55562028e-01
 -1.60256882e-01 -1.65625285e-01 -1.72447449e-01 -1.80994726e-01
 -1.91126351e-01 -2.05534356e-01 -2.27547189e-01 -2.56133344e-01
 -2.87934213e-01 -3.19555301e-01 -3.47355612e-01 -3.66308977e-01
 -3.71490987e-01 -3.59370201e-01 -3.27044532e-01 -2.74979379e-01
 -2.06100339e-01 -1.26483303e-01 -4.98319410e-02  7.84814965e-03
  3.70967032e-02  3.83326363